# 逻辑回归

## 1 Tensorflow模型

In [20]:
import tensorflow as tf
from __future__ import print_function
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./aaa/", one_hot=True)


Extracting ./aaa/train-images-idx3-ubyte.gz
Extracting ./aaa/train-labels-idx1-ubyte.gz
Extracting ./aaa/t10k-images-idx3-ubyte.gz
Extracting ./aaa/t10k-labels-idx1-ubyte.gz


In [21]:
print('Training Pic:',mnist.train.images.shape)
print('Training Label:',mnist.train.labels.shape)
print('Validation Pic:',mnist.validation.images.shape)
print('Validation Label:',mnist.validation.labels.shape)
print('Testing Pic:',mnist.test.images.shape)
print('Testing Label:',mnist.test.labels.shape)

Training Pic: (55000, 784)
Training Label: (55000, 10)
Validation Pic: (5000, 784)
Validation Label: (5000, 10)
Testing Pic: (10000, 784)
Testing Label: (10000, 10)


In [22]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [24]:
# Start training
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))

Epoch: 0001 cost= 1.183836952
Epoch: 0002 cost= 0.665236634
Epoch: 0003 cost= 0.552802216
Epoch: 0004 cost= 0.498660647
Epoch: 0005 cost= 0.465541036
Epoch: 0006 cost= 0.442595488
Epoch: 0007 cost= 0.425526262
Epoch: 0008 cost= 0.412193921
Epoch: 0009 cost= 0.401363390
Epoch: 0010 cost= 0.392402320
Epoch: 0011 cost= 0.384782241
Epoch: 0012 cost= 0.378195981
Epoch: 0013 cost= 0.372411460
Epoch: 0014 cost= 0.367341712
Epoch: 0015 cost= 0.362640563
Epoch: 0016 cost= 0.358609103
Epoch: 0017 cost= 0.354882024
Epoch: 0018 cost= 0.351479779
Epoch: 0019 cost= 0.348336937
Epoch: 0020 cost= 0.345432300
Epoch: 0021 cost= 0.342742997
Epoch: 0022 cost= 0.340268287
Epoch: 0023 cost= 0.337882960
Epoch: 0024 cost= 0.335713063
Epoch: 0025 cost= 0.333741117
Optimization Finished!
Accuracy: 0.888


In [25]:
with tf.summary.FileWriter(logdir='logs_linear_regression', graph=tf.get_default_graph()) as writer:
    writer.flush()

In [26]:
!tensorboard --logdir=logs_linear_regression

TensorBoard 0.4.0rc3 at http://1da2a2627c2e:6006 (Press CTRL+C to quit)
^C


## 2 tf.estimator 模型

In [43]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.learn import LinearClassifier
tf.logging.set_verbosity(tf.logging.ERROR)

image_size = 28
labels_size = 10
hidden_size = 1024

# Read in the MNIST dataset
mnist = input_data.read_data_sets("./aaa/", one_hot=False)

def input_fn(dataset):
    features = dataset.images
    labels = dataset.labels.astype(np.int32)
    return features, labels

# Define the Estimator
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=image_size*image_size)]
classifier = LinearClassifier(feature_columns=feature_columns,
                                              n_classes=labels_size,
                                              optimizer=tf.train.AdamOptimizer(),
                                              model_dir='estim')

# Fit the model
features, labels = input_fn(mnist.train)
classifier.fit(x=features, y=labels, batch_size=100, steps=1000)

# Evaluate the model on the test data
features, labels = input_fn(mnist.test)
test_accuracy = classifier.evaluate(x=features, y=labels, steps=1)["accuracy"]

print("\nTest accuracy: %g %%"%(test_accuracy*100))

Extracting ./aaa/train-images-idx3-ubyte.gz
Extracting ./aaa/train-labels-idx1-ubyte.gz
Extracting ./aaa/t10k-images-idx3-ubyte.gz
Extracting ./aaa/t10k-labels-idx1-ubyte.gz

Test accuracy: 91.39 %

Predicted labels from validation set: [3, 0, 4, 1, 9, 2, 1, 3, 1, 4]
Underlying values: [5, 0, 4, 1, 9, 2, 1, 3, 1, 4]


In [44]:
!tensorboard --logdir=estim

TensorBoard 0.4.0rc3 at http://1da2a2627c2e:6006 (Press CTRL+C to quit)
W0408 13:00:11.756196 Reloader plugin_event_accumulator.py:303] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0408 13:00:11.780200 Reloader plugin_event_accumulator.py:311] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
^C


## 3 tf.keras 模型

In [4]:
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.optimizers import SGD, RMSprop
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.callbacks import TensorBoard

batch_size = 128
num_classes = 10
epochs = 20

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

model = Sequential()  
model.add(Dense(units=10,input_shape=(784,)))  
model.add(Dense(num_classes, activation='softmax'))
sgd = SGD(lr=0.000001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', #sgd, #RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks = [TensorBoard(log_dir='keras-log2')])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



(60000, 'train samples')
(10000, 'test samples')
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s - loss: 0.6648 - acc: 0.8229 - val_loss: 0.3465 - val_acc: 0.9030
Epoch 2/20
60000/60000 [==============================] - 3s - loss: 0.3293 - acc: 0.9078 - val_loss: 0.2977 - val_acc: 0.9138
Epoch 3/20
60000/60000 [==============================] - 3s - loss: 0.2965 - acc: 0.9166 - val_loss: 0.2798 - val_acc: 0.9215
Epoch 4/20
60000/60000 [==============================] - 3s - loss: 0.2823 - acc: 0.9212 - val_loss: 0.2762 - val_acc: 0.9229
Epoch 5/20
60000/60000 [==============================] - 3s - loss: 0.2739 - acc: 0.9232 - val_loss: 0.2698 - val_acc: 0.9247
Epoch 6/20
60000/60000 [==============================] - 3s - loss: 0.2677 - acc: 0.9253 - val_loss: 0.2726 - val_acc: 0.9237
Epoch 7/20
60000/60000 [==============================] - 3s - loss: 0.2634 - acc: 0.9272 - val_loss: 0.2695 - val_acc: 0.9257
Epoch 8/20
6

In [5]:
!tensorboard --logdir=keras-log2

TensorBoard 0.4.0rc3 at http://1da2a2627c2e:6006 (Press CTRL+C to quit)
^C
